In [1]:
REGION = 'us-central1'

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [3]:

from google.cloud import artifactregistry_v1

In [4]:

ar_client = artifactregistry_v1.ArtifactRegistryClient()


List Repositories

In [5]:
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    print(repo.name)

projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-docker
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python


Get Repository

In [6]:
repo = ar_client.get_repository(name = f'projects/{PROJECT_ID}/locations/{REGION}/repositories/{PROJECT_ID}-python')

In [7]:
repo.name

'projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python'

List Packages In Repository

In [8]:
ar_client.list_packages(parent = repo.name)

ListPackagesPager<packages {
  name: "projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python/packages/tips-trainer"
  create_time {
    seconds: 1663768642
    nanos: 502115000
  }
  update_time {
    seconds: 1663768642
    nanos: 771288000
  }
}
>

List Files In Repository

In [10]:
ar_client.list_files(parent = repo.name)

ListFilesPager<files {
  name: "projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python/files/tips-trainer%2Ftips_trainer-0.1-py3-none-any.whl"
  size_bytes: 3626
  create_time {
    seconds: 1663768642
    nanos: 502115000
  }
  update_time {
    seconds: 1663768642
    nanos: 502115000
  }
  owner: "projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python/packages/tips-trainer/versions/0.1"
}
files {
  name: "projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python/files/tips-trainer%2Ftips_trainer-0.1.tar.gz"
  size_bytes: 3070
  create_time {
    seconds: 1663768642
    nanos: 771288000
  }
  update_time {
    seconds: 1663768642
    nanos: 771288000
  }
  owner: "projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python/packages/tips-trainer/versions/0.1"
}
>

List Files For Specific Repository

In [ ]:
ar_client.list_files(
    request = artifactregistry_v1.ListFilesRequest(
        parent = repo.name,
        filter = f"""name='{repo.name}/files/*'"""
    )
)

In [15]:
files = ar_client.list_files(parent = repo.name)

In [16]:
for file in files:
    print(file.name)

projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python/files/tips-trainer%2Ftips_trainer-0.1-py3-none-any.whl
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python/files/tips-trainer%2Ftips_trainer-0.1.tar.gz


In [22]:
files = list(files)

In [31]:
files[1].name.replace('%2F', '/').split('/')

['projects',
 'statmike-mlops-349915',
 'locations',
 'us-central1',
 'repositories',
 'statmike-mlops-349915-python',
 'files',
 'tips-trainer',
 'tips_trainer-0.1.tar.gz']

In [32]:
files[1].name.replace('%2F', '/').split('/')[-1]

'tips_trainer-0.1.tar.gz'

In [30]:
'/'.join(files[1].name.replace('%2F', '/').split('/')[5:])

'statmike-mlops-349915-python/files/tips-trainer/tips_trainer-0.1.tar.gz'

In [39]:
!curl -X GET \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-o {files[1].name.split('/')[-1]} \
-L https://{REGION}-python.pkg.dev/{PROJECT_ID}/{'/'.join(files[1].name.split('/')[5:])}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1449  100  1449    0     0  46584      0 --:--:-- --:--:-- --:--:-- 46741


In [36]:
print(f"https://{REGION}-python.pkg.dev/{PROJECT_ID}/{'/'.join(files[1].name.split('/')[5:])}")

https://us-central1-python.pkg.dev/statmike-mlops-349915/statmike-mlops-349915-python/files/tips-trainer%2Ftips_trainer-0.1.tar.gz


In [42]:
repo.name

'projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python'

In [43]:
!curl -X GET \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
https://artifactregistry.googleapis.com/v1/{repo.name}/pythonPackages

{
  "pythonPackages": [
    {
      "name": "projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python/pythonPackages/tips-trainer:0.1",
      "packageName": "tips-trainer",
      "version": "0.1",
      "createTime": "2022-09-21T13:57:22.502115Z",
      "updateTime": "2022-09-21T13:57:22.771288Z"
    }
  ]
}


In [ ]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict